# Objects Detection

## Tasks

### Task 1

Calculate the `Jaccard similarity` for class 0, class 1, and class 2.  
Calculate the average and round to 4 decimal places.

In [1]:
import torch
from torch import Tensor


y_pred = Tensor([
    [1, 1, 2, 2, 2],
    [1, 1, 2, 1, 2],
    [1, 0, 0, 0, 0],
    [2, 2, 2, 0, 0],
    [2, 1, 1, 1, 2]
])

y_true = Tensor([
    [1, 1, 1, 2, 2],
    [1, 1, 1, 2, 2],
    [1, 1, 1, 2, 2],
    [0, 0, 0, 2, 2],
    [0, 0, 0, 2, 2]
])

# Flatten the tensors
y_pred_flat = y_pred.flatten()
y_true_flat = y_true.flatten()

# Calculate Jaccard similarity for class 0
pred_class_0 = (y_pred_flat == 0)
true_class_0 = (y_true_flat == 0)
intersection_0 = (pred_class_0 & true_class_0).sum().item()
union_0 = (pred_class_0 | true_class_0).sum().item()
jaccard_0 = intersection_0 / union_0 if union_0 > 0 else 0

# Calculate Jaccard similarity for class 1
pred_class_1 = (y_pred_flat == 1)
true_class_1 = (y_true_flat == 1)
intersection_1 = (pred_class_1 & true_class_1).sum().item()
union_1 = (pred_class_1 | true_class_1).sum().item()
jaccard_1 = intersection_1 / union_1 if union_1 > 0 else 0

# Calculate Jaccard similarity for class 2
pred_class_2 = (y_pred_flat == 2)
true_class_2 = (y_true_flat == 2)
intersection_2 = (pred_class_2 & true_class_2).sum().item()
union_2 = (pred_class_2 | true_class_2).sum().item()
jaccard_2 = intersection_2 / union_2 if union_2 > 0 else 0

# Calculate the average
average_jaccard = (jaccard_0 + jaccard_1 + jaccard_2) / 3

In [2]:
print(f'Class 0 - Intersection: {intersection_0}, Union: {union_0}, Jaccard: {jaccard_0:.4f}')
print(f'Class 1 - Intersection: {intersection_1}, Union: {union_1}, Jaccard: {jaccard_1:.4f}')
print(f'Class 2 - Intersection: {intersection_2}, Union: {union_2}, Jaccard: {jaccard_2:.4f}')
print(f'Average Jaccard similarity: {average_jaccard:.4f}')

Class 0 - Intersection: 0, Union: 12, Jaccard: 0.0000
Class 1 - Intersection: 5, Union: 13, Jaccard: 0.3846
Class 2 - Intersection: 4, Union: 16, Jaccard: 0.2500
Average Jaccard similarity: 0.2115
